In [2]:
import pandas as pd
from pathlib import Path
import json

from math import log
from numpy import random
from datetime import date

In [5]:
def getDistinctMakes(carData):
    makes=[]
    for make in carData[['make', 'model']].values:
        if tuple(make) not in makes:
            makes.append(tuple(make))
    return makes

In [60]:
path = Path.cwd()
path = path.joinpath('Models')

modelPaths = list(path.glob('*'))
model = 'HuberRegressor_202209131836'

testResults = pd.DataFrame()
CVResults = pd.DataFrame()
for modelPath in modelPaths:
    if modelPath.stem == model:

        testPath = modelPath.joinpath('testResults.csv')
        testResult = pd.read_csv(str(testPath))

        CVPath = modelPath.joinpath('CV.csv')
        CVResult = pd.read_csv(str(CVPath))

        CFGPath = modelPath.joinpath('CFG.txt')
        CFG = {}
        with CFGPath.open() as f:
            CFG = json.loads(f.read())
        for key in CFG:
            testResult[key] = str(CFG[key])
            if key=='notes':
                CVResult[key] = str(CFG[key])


        if len(testResults)==0:
            testResults = testResult.copy()
            CVResults = CVResult.copy()
        else:
            testResults = pd.concat([testResults, testResult])
            CVResults = pd.concat([CVResults, CVResult])


In [40]:
makeModel = getDistinctMakes(testResult)

In [56]:
alphas = {}
for make in testResult['make'].unique():
    alphas[make]={}
    for model in testResult['model'].unique():
        view = testResult[(testResult['make']==make) & (testResult['model']==model)]
        if len(view)>0:
            value = view['alpha'].max()
            values = [value + eps/10 for eps in range(-10, 11) if value+eps/10 >=0]
            alphas[make][model] = values
alphas = {'alpha': alphas}

In [57]:
with open('alpha.txt', 'w') as f:
    f.write(json.dumps(alphas))

In [59]:
with open('toTrain.txt', 'w') as f:
    f.write(json.dumps([('Seat', 'Leon')]))
